# Environment

In [16]:
import numpy as np, pandas as pd, tensorflow as tf

from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession

# Hyperparameters

In [13]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 10

# BigQuery Connector

Documentation: https://github.com/tensorflow/io/tree/master/tensorflow_io/bigquery

In [14]:
GCP_PROJECT_ID = 'axa-ch-machine-learning-dev'
DATASET_GCP_PROJECT_ID = 'axa-ch-machine-learning-dev'
DATASET_ID = 'nlp_text_classification'
TABLE_ID = 'stackoverflow_posts'

In [18]:
client = BigQueryClient()

read_session = client.read_session(
    "projects/" + GCP_PROJECT_ID,
    DATASET_GCP_PROJECT_ID,
    TABLE_ID,
    DATASET_ID,
    ['body', 'tags'],
    [dtypes.string, dtypes.string],
    row_restriction="",
)
dataset = read_session.parallel_read_rows() \
            .shuffle(buffer_size=BUFFER_SIZE) \
            .batch(batch_size=BATCH_SIZE)

A tf.data dataset. Should return a tuple of either (inputs, targets) or (inputs, targets, sample_weights). <br>
Problem: the dataset above does not return a tuple of inputs and labels

# Feature Layer

In [10]:
feature_names = ['title', 'body', 'tags']

In [11]:
feature_columns = []

for name in feature_names:
    feature_columns.append(tf.feature_column.numeric_column(name))

In [19]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

# Model

In [23]:
n_cols = 10 #number of features
n_labels = 10 #number of labels

In [21]:
model = tf.keras.Sequential(
    [
        feature_layer,
        tf.keras.layers.Dense(n_cols, activation="relu", input_shape(n_cols,)),
        tf.keras.layers.Dense(128, activation="relu"), #number of neurons must be determined first
        tf.keras.layers.Dense(n_labels, activation="sigmoid"),
    ]
)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Parameters: <a href src='https://www.tensorflow.org/api_docs/python/tf/keras/Model' target='_blank'>model.fit</a>

# Fit Model (Non-Dataset Input)

In [ ]:
model.fit(
    x=None,
    y=None,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_split=0.2,
    shuffle=True,
)

# Fit Model (Dataset Input)

In [ ]:
model.fit(dataset, verbose=2)

# Evaluate Model

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

https://towardsdatascience.com/multi-label-image-classification-with-neural-network-keras-ddc1ab1afede